In [36]:
pip install sktime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.2/118.2 kB 13.7 MB/s eta 0:00:00


In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np
import sktime
from sktime.forecasting.model_selection import temporal_train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/Data/power

/content/drive/MyDrive/Data/power


In [20]:
train = pd.read_csv("pre_train.csv", index_col = 0)
test = pd.read_csv("pre_test.csv", index_col = 0)
train

,building_number,temperature,windspeed,humidity,power_consumption,day,month,day_of_week,cos_time,sin_time,...,day_hour_mean,hour_mean,hour_std,discomfort,CDH,building_type,total_area,cooling_area,solar_power_capacity,ess_capacity
0,1,18.6,0.9,42.0,1085.28,1,6,2,1.000000,0.000000,...,1371.52,1706.318118,446.882767,63.09388,-7.4,Other Buildings,110634.00,39570.00,0.0,0.0
1,1,18.0,1.1,45.0,1047.36,1,6,2,0.965926,0.258819,...,1323.52,1622.620235,439.662704,62.46400,-15.4,Other Buildings,110634.00,39570.00,0.0,0.0
2,1,17.7,1.5,45.0,974.88,1,6,2,0.866025,0.500000,...,1236.48,1506.971294,412.071906,62.08735,-23.7,Other Buildings,110634.00,39570.00,0.0,0.0
3,1,16.7,1.4,48.0,953.76,1,6,2,0.707107,0.707107,...,1189.92,1437.365647,391.205981,60.89884,-33.0,Other Buildings,110634.00,39570.00,0.0,0.0
4,1,18.4,2.8,43.0,986.40,1,6,2,0.500000,0.866025,...,1210.08,1447.321412,381.099697,62.88788,-40.6,Other Buildings,110634.00,39570.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,23.1,0.9,86.0,881.04,24,8,2,0.258819,-0.965926,...,942.96,1010.462118,161.399578,72.38034,-19.6,Hotel and Resort,57497.84,40035.23,0.0,0.0
203996,100,22.4,1.3,86.0,798.96,24,8,2,0.500000,-0.866025,...,855.84,928.125176,137.566008,71.21736,-20.2,Hotel and Resort,57497.84,40035.23,0.0,0.0
203997,100,21.3,1.0,92.0,825.12,24,8,2,0.707107,-0.707107,...,809.20,830.032941,128.300189,69.79704,-22.3,Hotel and Resort,57497.84,40035.23,0.0,0.0
203998,100,21.0,0.3,94.0,640.08,24,8,2,0.866025,-0.500000,...,696.08,723.100235,112.464079,69.41060,-25.1,Hotel and Resort,57497.84,40035.23,0.0,0.0


# 테스트 1
---

In [21]:
train_1 = train
test_1 = test

# 카테고리 컬럼 설정
train_1[['building_type']] = train_1[[ 'building_type']].astype('category')
test_1[['building_type']] = test_1[['building_type']].astype('category')

train_1 = pd.get_dummies(train_1, columns = ['building_type'])

# 특성과 라벨 분리
X_train_1 = train_1.drop(columns = ['power_consumption'])
y_train_1 = train_1['power_consumption']



In [24]:
# 데이터 정규화
scaler = StandardScaler()
X_train_scaled_1 = scaler.fit_transform(X_train_1)

# 데이터 분할
X_train_split_1, X_test_1, y_train_split_1, y_test_1 = train_test_split(X_train_scaled_1, y_train_1, random_state=42)

# 하이퍼파라미터 그리드 정의
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'n_estimators': [5000],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9],
}

# XGBoost 모델 생성
xgb_model_1 = XGBRegressor(objective='reg:squarederror', random_state=42)

def smape(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output

xgb_model_1.fit(X_train_1, y_train_1)

y_pred_1 = xgb_model_1.predict(X_test_1)
print(f"Test #1: {smape(y_test_1, y_pred_1)}")

Test #1: 106.27355878162138


# 테스트 2
---
건물별로 모델링 해보기

In [42]:
train_2 = train

# 카테고리 컬럼 설정
train_2[['building_type']] = train_2[[ 'building_type']].astype('category')

train_2 = pd.get_dummies(train_2, columns = ['building_type'])

# 특성과 라벨 분리
# X_train_2 = train_2.drop(columns = ['power_consumption'])
# y_train_2 = train_2['power_consumption']
y = train_2.loc[train['building_number'] == 1, 'power_consumption']
x = train_2.loc[train['building_number'] == 1].drop(columns = 'power_consumption')

In [43]:
x.columns

Index(['building_number', 'temperature', 'windspeed', 'humidity', 'day',
       'month', 'day_of_week', 'cos_time', 'sin_time', 'holiday',
       'day_hour_mean', 'hour_mean', 'hour_std', 'discomfort', 'CDH',
       'total_area', 'cooling_area', 'solar_power_capacity', 'ess_capacity',
       'building_type_Apartment', 'building_type_Commercial',
       'building_type_Data Center',
       'building_type_Department Store and Outlet',
       'building_type_Discount Mart', 'building_type_Hospital',
       'building_type_Hotel and Resort',
       'building_type_Knowledge Industry Center',
       'building_type_Other Buildings', 'building_type_Public',
       'building_type_Research Institute', 'building_type_University'],
      dtype='object')

In [45]:
x

,building_number,temperature,windspeed,humidity,day,month,day_of_week,cos_time,sin_time,holiday,...,building_type_Data Center,building_type_Department Store and Outlet,building_type_Discount Mart,building_type_Hospital,building_type_Hotel and Resort,building_type_Knowledge Industry Center,building_type_Other Buildings,building_type_Public,building_type_Research Institute,building_type_University
0,1,18.6,0.9,42.0,1,6,2,1.000000,0.000000,False,...,0,0,0,0,0,0,1,0,0,0
1,1,18.0,1.1,45.0,1,6,2,0.965926,0.258819,False,...,0,0,0,0,0,0,1,0,0,0
2,1,17.7,1.5,45.0,1,6,2,0.866025,0.500000,False,...,0,0,0,0,0,0,1,0,0,0
3,1,16.7,1.4,48.0,1,6,2,0.707107,0.707107,False,...,0,0,0,0,0,0,1,0,0,0
4,1,18.4,2.8,43.0,1,6,2,0.500000,0.866025,False,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035,1,25.5,2.9,63.0,24,8,2,0.258819,-0.965926,False,...,0,0,0,0,0,0,1,0,0,0
2036,1,25.0,2.6,66.0,24,8,2,0.500000,-0.866025,False,...,0,0,0,0,0,0,1,0,0,0
2037,1,24.6,1.3,67.0,24,8,2,0.707107,-0.707107,False,...,0,0,0,0,0,0,1,0,0,0
2038,1,24.1,2.1,70.0,24,8,2,0.866025,-0.500000,False,...,0,0,0,0,0,0,1,0,0,0


In [46]:
def weighted_mse(alpha = 1):
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual>0, -2*alpha*residual, -2*residual)
        hess = np.where(residual>0, 2*alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

In [52]:
# 데이터 정규화
scaler = StandardScaler()
X_train_scaled_2 = scaler.fit_transform(x)

# 데이터 분할
# X_train_split_2, X_test_2, y_train_split_2, y_test_2 = train_test_split(X_train_scaled_2, y, random_state=42)
y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = y, X = x, test_size = 168)

# # 하이퍼파라미터 그리드 정의
# param_grid = {
#     'learning_rate': [0.01, 0.05, 0.1],
#     'max_depth': [3, 5, 7],
#     'n_estimators': [5000],
#     'subsample': [0.7, 0.9],
#     'colsample_bytree': [0.7, 0.9],
# }

# XGBoost 모델 생성
xgb_model_2 = XGBRegressor(n_estimators = 10000, eta = 0.01, min_child_weight = 6,
                       max_depth = 5, colsample_bytree = 0.8,
                       subsample = 0.9, seed=0)
#xgb_model_2.set_params(**{'objective':weighted_mse(100)})

def smape(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output

xgb_model_2.fit(x_train, y_train, eval_set = [(x_train, y_train), (x_valid, y_valid)],
                early_stopping_rounds = 300, verbose = False)

pred = xgb_model_2.predict(x_valid)
print(f"[Test #2-1] SMAPE: {smape(y_valid, pred)}")

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[Test #2-1] SMAPE: 5.848202848220082
